In [2]:
from relapse_prediction import constants

from pathlib import Path

In [6]:
constants.L_CERCARE_MAPS

['COV', 'CTH', 'Delay', 'rCBV', 'rLeakage', 'OEF', 'rCMRO2']

In [7]:
constants.L_IRM_MAPS

['T1CE', 'T1', 'FLAIR']